In [1]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd


In [6]:
dx = pd.read_csv(r'/home/nkem/Documents/PhD_Research/allN11Oct2022.csv')
dx['incidentdate'] = pd.to_datetime(dx['incidentdate'])
td = dx.copy()
dk = td.groupby([pd.Grouper(key='incidentdate', freq='M')])['estimatedqty'].agg(['sum','size'])
dk = dk.reset_index()
dk.rename(columns={"sum":"estimatedqty", "size":"spillno"}, inplace=True)

df = dk[["incidentdate","spillno"]]
df = df.set_index("incidentdate")
df = df.drop(index="2022-10-31")

df= df.reset_index()
df["month"] = df["incidentdate"].dt.month
df["year"] = df["incidentdate"].dt.year
dt = df[["incidentdate","spillno","month", "year"]]
dt = dt.set_index("incidentdate") 

In [ ]:
import statsmodels.api as sm
import pandas as pd
#data = sm.datasets.co2.load_pandas().data

dx = pd.read_csv(r'/home/nkem/Documents/PhD_Research/dlam.csv')
dx['incidentdate'] = pd.to_datetime(dx['incidentdate'])

# data is given in weeks, but the task is to predict monthly, so use monthly averages instead
#data = data['co2'].resample('MS').mean()
#data = data.bfill().ffill()  # makes sure there are no missing values
#data = data.to_frame().reset_index()
df = dx[["incidentdate","spillno"]]
df = df.rename(columns={"incidentdate":"index"})

In [10]:
end_train = '2022-03-31'
start_test = '2022-07-30'
end_test = '2022-09-30'

data_train = dt.loc[: end_train, :]
data_test  = dt.loc[start_test: end_test]

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}")

Train dates      : 2005-01-31 00:00:00 --- 2022-03-31 00:00:00
Test dates       : 2022-07-31 00:00:00 --- 2022-09-30 00:00:00


In [11]:
# Select exogenous variables, including those generated by one hot encoding.
exog_variables = [column for column in dt.columns
                      if column.startswith(('year', 'month'))]
#exog_variables.extend(['estimatedqty'])
#print(exog_variables)

In [12]:
train_x = data_train[exog_variables]
train_y = data_train["spillno"]
test_x = data_test[exog_variables]
test_y = data_test["spillno"]